# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'
df_fold = f'{drive}:/data/big2/BD1/df'

# Load
- 등록&제원 -> result 4등급(연료 : 경유, 휘발유, LPG)
- 정기/정밀검사 -> 연간 주행거리(1일평균주행거리)

## 말소현황

In [4]:
# about 19.3s
name = '말소정보_전처리완료'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642216 entries, 0 to 3642215
Data columns (total 32 columns):
 #   Column       Dtype  
---  ------       -----  
 0   차량관리번호       object 
 1   본거지법정동코드     int64  
 2   차종_등록정보      object 
 3   차대번호         object 
 4   최초등록일자       int64  
 5   차량연식         int64  
 6   제원관리번호       object 
 7   배출가스인증번호     object 
 8   배출가스등급       object 
 9   차량말소YN_등록정보  object 
 10  차명           object 
 11  차종유형(소분류)    object 
 12  제작사명         object 
 13  연료           object 
 14  등급           object 
 15  차량말소YN_변경이력  object 
 16  변경일자         int64  
 17  변경구분코드       float64
 18  법정동코드        int64  
 19  시도           object 
 20  시구군          object 
 21  읍면동          object 
 22  리            object 
 23  지역3          float64
 24  폐지여부         object 
 25  대기관리권역       object 
 26  대기관리권역YN     object 
 27  저감장치         object 
 28  변경일자_년       int64  
 29  변경일자_월       int64  
 30  변경일자_일       int64  
 31  변경일자_년_월     float64
dty

## 정기, 정밀 검사(최근 3년)

In [5]:
# # about 5m 5.3s
# ins = pd.read_csv(os.path.join(ods_fold, '[ODS]정기&정밀검사(최근3년)(2022.12.28).csv'), encoding='cp949')
# ins.info()

# 전처리

In [6]:
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종_등록정보', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형(소분류)', '제작사명', '연료',
       '등급', '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [7]:
df = df.rename(columns={'차종_등록정보':'차종', '차종유형(소분류)':'차종유형'})
df.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [8]:
df['연료'].value_counts(dropna=False)

연료
경유             1660076
휘발유            1528227
LPG(액화석유가스)     412084
휘발유 하이브리드        14497
CNG(압축천연가스)       9204
전기                8018
NaN               7164
LPG 하이브리드         2662
기타연료               118
수소                  99
경유 하이브리드            43
CNG 하이브리드           22
알코올                  1
태양열                  1
Name: count, dtype: int64

In [9]:
df['등급'].value_counts(dropna=False)

등급
3.0    1198403
5.0    1189945
2.0     783791
4.0     366939
1.0      88793
X        14313
NaN         32
Name: count, dtype: int64

## 4등급 차량(휘발유, 경유, LPG) 추출

In [10]:
df1 = df[df['등급'] == '4.0'].reset_index(drop=True)
df1.shape

(366939, 32)

In [11]:
df1['연료'].value_counts(dropna=False)

연료
경유             229930
휘발유            124839
LPG(액화석유가스)     12142
NaN                17
CNG(압축천연가스)        10
기타연료                1
Name: count, dtype: int64

In [12]:
etc = df1[(df1['연료'] == '알코올') | (df1['연료'] == 'CNG(압축천연가스)') | (df1['연료'] == '기타연료') | (df1['연료'].isnull() == True)].reset_index(drop=True)
etc.shape

(28, 32)

In [13]:
df1[(df1['연료'] == '휘발유') | (df1['연료'] == '경유') | (df1['연료'] == 'LPG(액화석유가스)')].shape

(366911, 32)

In [14]:
df2 = df1[(df1['연료'] == '휘발유') | (df1['연료'] == '경유') | (df1['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
df2.shape

(366911, 32)

In [15]:
df2.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [16]:
df2['연료'].value_counts(dropna=False)

연료
경유             229930
휘발유            124839
LPG(액화석유가스)     12142
Name: count, dtype: int64

## 중복 차대번호 확인
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [17]:
df2.shape[0], len(df2['차대번호'].unique())

(366911, 366911)

In [18]:
df2['차종'].value_counts(dropna=False)

차종
승용    261481
화물     72295
승합     32407
특수       728
Name: count, dtype: int64

In [19]:
df2['차종유형'].value_counts()

차종유형
중형    215434
소형    108708
대형     32888
경형      9881
Name: count, dtype: int64

In [20]:
df2['차량연식'].describe()

count   366911.00
mean      2004.69
std          5.40
min       1987.00
25%       2000.00
50%       2007.00
75%       2009.00
max       2018.00
Name: 차량연식, dtype: float64

## 연료별 차종별 차종유형 확인

In [21]:
# df1.groupby(['연료', '차종', '차종유형'])['차대번호'].count()

In [22]:
for fuel in ['경유', '휘발유', 'LPG(액화석유가스)']:
    temp1 = df2[df2['연료'] == fuel].reset_index(drop=True)
    for car_type in temp1['차종'].unique():
        temp2 = temp1[temp1['차종'] == car_type].reset_index(drop=True)
        car_size = list(temp2['차종유형'].unique())
        print(f'{fuel} - {car_type}의 차종유형 : {car_size}')

경유 - 승용의 차종유형 : ['소형', '중형', '대형']
경유 - 화물의 차종유형 : ['소형', '중형', '대형']
경유 - 승합의 차종유형 : ['중형', '대형']
경유 - 특수의 차종유형 : ['중형', '대형', '소형']
휘발유 - 승용의 차종유형 : ['중형', '소형', '경형', '대형']
휘발유 - 승합의 차종유형 : ['중형', '경형', '소형', '대형']
휘발유 - 화물의 차종유형 : ['경형', '소형', '중형', '대형']
휘발유 - 특수의 차종유형 : ['중형']
LPG(액화석유가스) - 화물의 차종유형 : ['경형', '소형', '중형']
LPG(액화석유가스) - 승용의 차종유형 : ['중형', '대형', '경형', '소형']
LPG(액화석유가스) - 승합의 차종유형 : ['중형', '경형', '소형']


### 차령 정보 계산 및 데이터 추가

In [23]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [24]:
df2.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월'],
      dtype='object')

In [25]:
df2['차령'] = current_yr - df2['차량연식']
df2['차령'].describe()

count   366911.00
mean        18.31
std          5.40
min          5.00
25%         14.00
50%         16.00
75%         23.00
max         36.00
Name: 차령, dtype: float64

## 시도명, 시군구명 수정
- 시군구명 앞지역명만 추출
- 세종특별자치시 -> 세종시
- 여주군 -> 여주시
- 청원군 -> 청원시
- 인천광역시 남구 -> 미추홀구

In [26]:
df2[['시도', '시구군', '차종']].isnull().sum()

시도     0
시구군    0
차종     0
dtype: int64

In [27]:
df2['시도'].unique()

array(['경기도', '인천광역시', '제주특별자치도', '서울특별시', '부산광역시', '충청북도', '대전광역시',
       '울산광역시', '경상남도', '전라남도', '대구광역시', '세종특별자치시', '광주광역시', '충청남도',
       '전라북도', '경상북도', '강원도'], dtype=object)

In [28]:
df2[['시도', '시구군']].iloc[10:20]

,시도,시구군
10,대전광역시,동구
11,서울특별시,서대문구
12,경기도,고양시 일산서구
13,울산광역시,남구
14,경상남도,창원시 마산회원구
15,경기도,화성시
16,경기도,양평군
17,서울특별시,강남구
18,서울특별시,구로구
19,서울특별시,강서구


In [29]:
df2['시구군_수정'] = df2['시구군'].str.split(' ').str[0]

In [30]:
df2[['시도', '시구군', '시구군_수정']].iloc[10:20]

,시도,시구군,시구군_수정
10,대전광역시,동구,동구
11,서울특별시,서대문구,서대문구
12,경기도,고양시 일산서구,고양시
13,울산광역시,남구,남구
14,경상남도,창원시 마산회원구,창원시
15,경기도,화성시,화성시
16,경기도,양평군,양평군
17,서울특별시,강남구,강남구
18,서울특별시,구로구,구로구
19,서울특별시,강서구,강서구


In [31]:
df2.loc[df2['시구군_수정'] == '세종특별자치시', '시구군_수정'].shape

(0,)

In [32]:
# df2.loc[df2['시구군_수정'] == '세종특별자치시', '시구군_수정'] = '세종시'

In [33]:
# df2.loc[df2['시구군_수정'] == '세종특별자치시', '시구군_수정'].shape

In [34]:
df2.loc[df2['시구군_수정'] == '여주군', '시구군_수정'].shape

(0,)

In [35]:
# df2.loc[df2['시구군_수정'] == '여주군', '시구군_수정'] = '여주시'

In [36]:
# df2.loc[df2['시구군_수정'] == '여주군', '시구군_수정'].shape

In [37]:
df2.loc[df2['시구군_수정'] == '청원군', '시구군_수정'].shape

(0,)

In [38]:
# df2.loc[df2['시구군_수정'] == '청원군', '시구군_수정'] = '청주시'

In [39]:
# df2.loc[df2['시구군_수정'] == '청원군', '시구군_수정'].shape

In [40]:
df2.loc[(df2['시도'] == '인천광역시') & (df2['시구군_수정'] == '남구'), '시구군_수정'].shape

(0,)

In [41]:
# df2.loc[(df2['시도'] == '인천광역시') & (df2['시구군_수정'] == '남구'), '시구군_수정'] = '미추홀구'

In [42]:
# df2.loc[(df2['시도'] == '인천광역시') & (df2['시구군_수정'] == '남구'), '시구군_수정'].shape

### [로드] 1일평균주행거리(2021)

In [43]:
# 1일 평균 주행거리 자료 로드
run = pd.read_excel(os.path.join(raw_fold, '용도별_차종별_시군구별_자동차1일평균주행거리(2021).xlsx'))
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


### [병합] 1일평균주행거리 데이터 추가(df1)

In [44]:
df3 = df2.merge(run, left_on=['시도', '시구군_수정', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
df3.shape

(366911, 37)

In [45]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '전체'],
      dtype='object')

In [46]:
df3['전체'].isnull().sum()

3555

In [47]:
df3['차종'].isnull().sum()

0

In [48]:
df3.loc[df3['전체'].isnull() == True, ['시도', '시구군_수정', '차종', '시도명', '시군구명']].head()

,시도,시구군_수정,차종,시도명,시군구명
27,세종특별자치시,대평동,승용,NaN,NaN
60,세종특별자치시,연기면,화물,NaN,NaN
76,부산광역시,진구,화물,NaN,NaN
135,세종특별자치시,보람동,화물,NaN,NaN
212,세종특별자치시,전의면,승용,NaN,NaN


In [49]:
dis_y = df3.loc[df3['전체'].isnull() == False].reset_index(drop=True)
dis_n = df3.loc[df3['전체'].isnull() == True].reset_index(drop=True)
dis_y.shape, dis_n.shape

((363356, 37), (3555, 37))

In [50]:
dis_n.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '전체'],
      dtype='object')

In [51]:
dis_n = dis_n.drop(['시도명', '시군구명', '전체'], axis=1)
dis_n.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정'],
      dtype='object')

In [52]:
run.shape

(1230, 4)

In [53]:
run_sort = run.drop_duplicates(['시도명', '차종']).reset_index(drop=True)
run_sort.shape

(85, 4)

In [54]:
dis_n.shape

(3555, 34)

In [55]:
dis_n = dis_n.merge(run_sort, left_on=['시도', '차종'], right_on=['시도명', '차종'], how='left')
dis_n.shape

(3555, 37)

In [56]:
dis_n.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '전체'],
      dtype='object')

In [57]:
dis_n['전체'].isnull().sum()

0

In [58]:
df3.shape

(366911, 37)

In [59]:
df3 = pd.concat([dis_y, dis_n], ignore_index=True)
df3.shape

(366911, 37)

In [60]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '전체'],
      dtype='object')

In [61]:
df3 = df3.rename(columns={'전체':'1일평균주행거리'})
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '1일평균주행거리'],
      dtype='object')

In [62]:
df3['1일평균주행거리'].isnull().sum()

0

In [63]:
df3['저감장치'].value_counts(dropna=False)

저감장치
NaN    366895
DPF        16
Name: count, dtype: int64

# 자동차-엔진 가열(Hot-start) 배출

## 배출량 식
$$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

## 주행거리(VKT)
- [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

In [65]:
df3['VKT'] = df3['1일평균주행거리'] * 365

In [66]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '1일평균주행거리', 'VKT'],
      dtype='object')

In [67]:
df3[['시도', '시군구명', '차종', 'VKT']].head()

,시도,시군구명,차종,VKT
0,경기도,성남시,승용,12702.00
1,인천광역시,미추홀구,화물,17447.00
2,경기도,용인시,승용,13249.50
3,경기도,평택시,화물,20002.00
4,제주특별자치도,제주시,화물,14162.00


In [68]:
df3['VKT'].isnull().sum()

0

## DF(열화계수)
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
- 조건 수정(2023.04.20, 최)
    - 특수 소형 = 승용 소형
    - 승합 대형 = 화물 대형

In [69]:
df3['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)'], dtype=object)

In [70]:
DF_col = ['연료', '차종', '차종유형', '차량연식', '차령']

In [71]:
df3[DF_col].head()

,연료,차종,차종유형,차량연식,차령
0,경유,승용,소형,2007,16
1,경유,화물,소형,2011,12
2,경유,승용,중형,2008,15
3,경유,화물,소형,2009,14
4,경유,화물,소형,2008,15


In [72]:
df3[DF_col].values[0]

array(['경유', '승용', '소형', 2007, 16], dtype=object)

### ❗ DF(열화계수) 코드
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1

In [73]:
# about 5.0s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '차량연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df3[DF_col].values):    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG(액화석유가스)'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

100%|██████████| 366911/366911 [00:01<00:00, 283067.85it/s]


In [74]:
df3.shape[0]

366911

In [75]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(366911, 366911, 366911, 366911)

In [76]:
df3['DF_CO'] = DF_CO_list
df3['DF_HC'] = DF_HC_list
df3['DF_PM'] = DF_PM_list
df3['DF_NOx'] = DF_NOx_list

In [77]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df3.loc[df3['차령'] < 10, DF_col + check_DF_col].head()

,연료,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
1001,경유,승용,대형,2015,8,1.15,1.15,1.15,1.06
24556,경유,특수,대형,2018,5,1.00,1.00,1.00,1.00
55794,경유,화물,대형,2014,9,1.20,1.20,1.20,1.08
66590,경유,특수,대형,2017,6,1.05,1.05,1.05,1.02
69840,경유,특수,대형,2017,6,1.05,1.05,1.05,1.02


## 저감장치 부착 효율(R) 계산
- DPF유무_수정 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

### 현대자동차 상용차 중 pDPF 달린 차량
- 6MY-HD-23-55
- 7MY-HD-24-28
- 7MY-HD-24-30

In [78]:
df3['저감장치'].value_counts(dropna=False)

저감장치
NaN    366895
DPF        16
Name: count, dtype: int64

In [79]:
df3.loc[df3['저감장치'].isnull() == False, 'DPF유무_수정'] = '유'

In [80]:
df3['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
NaN    366895
유          16
Name: count, dtype: int64

In [81]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '1일평균주행거리', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'DPF유무_수정'],
      dtype='object')

In [82]:
df3.loc[(df3['배출가스인증번호'] == '6MY-HD-23-55') & (df3['제작사명'] == '현대자동차(주)')].shape

(0, 43)

In [83]:
df3.loc[(df3['배출가스인증번호'] == '7MY-HD-24-28') & (df3['제작사명'] == '현대자동차(주)')].shape

(2238, 43)

In [84]:
df3.loc[(df3['배출가스인증번호'] == '7MY-HD-24-30') & (df3['제작사명'] == '현대자동차(주)')].shape

(533, 43)

In [85]:
df3.loc[df3['DPF유무_수정'] == '유', ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6
df3.loc[(df3['DPF유무_수정'] == '유') & (df3['배출가스인증번호'] == '6MY-HD-23-55') & (df3['제작사명'] == '현대자동차(주)'), ['R_CO', 'R_HC', 'R_PM']] = 94.6, 89.3, 56
df3.loc[(df3['DPF유무_수정'] == '유') & (df3['배출가스인증번호'] == '7MY-HD-24-28') & (df3['제작사명'] == '현대자동차(주)'), ['R_CO', 'R_HC', 'R_PM']] = 94.6, 89.3, 56
df3.loc[(df3['DPF유무_수정'] == '유') & (df3['배출가스인증번호'] == '7MY-HD-24-30') & (df3['제작사명'] == '현대자동차(주)'), ['R_CO', 'R_HC', 'R_PM']] = 94.6, 89.3, 56

In [86]:
df3.loc[(df3['DPF유무_수정'] == '유') & (df3['배출가스인증번호'] == '6MY-HD-23-55') & (df3['제작사명'] == '현대자동차(주)'), 'R_PM'].head()

Series([], Name: R_PM, dtype: float64)

In [87]:
df3.loc[(df3['DPF유무_수정'] == '유') & (df3['배출가스인증번호'] == '7MY-HD-24-28') & (df3['제작사명'] == '현대자동차(주)'), 'R_PM'].head()

97987   56.00
Name: R_PM, dtype: float64

In [88]:
df3.loc[(df3['DPF유무_수정'] == '유') & (df3['배출가스인증번호'] == '7MY-HD-24-30') & (df3['제작사명'] == '현대자동차(주)'), 'R_PM'].head()

Series([], Name: R_PM, dtype: float64)

In [89]:
df3.shape[0]

366911

In [90]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']

In [91]:
df3[check_R_col].notnull().sum()

R_CO    16
R_HC    16
R_PM    16
dtype: int64

In [92]:
df3.loc[df3['DPF유무_수정'] == '유', DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
3926,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90.00,83.60
48854,경유,화물,소형,2008,15,1.50,1.50,1.50,1.20,99.50,90.00,83.60
52298,경유,승합,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90.00,83.60
72285,경유,승합,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90.00,83.60
97987,경유,화물,중형,2008,15,1.50,1.50,1.50,1.20,94.60,89.30,56.00


In [93]:
df3.loc[df3['DPF유무_수정'].isnull() == True, DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
0,경유,승용,소형,2007,16,1.50,1.50,1.50,1.20,NaN,NaN,NaN
1,경유,화물,소형,2011,12,1.35,1.35,1.35,1.14,NaN,NaN,NaN
2,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,NaN,NaN,NaN
3,경유,화물,소형,2009,14,1.45,1.45,1.45,1.18,NaN,NaN,NaN
4,경유,화물,소형,2008,15,1.50,1.50,1.50,1.20,NaN,NaN,NaN


### DPF 무, 확인불가, nan의 R값 수정
- 0

In [94]:
df3[check_R_col] = df3[check_R_col].fillna(0)

In [95]:
df3[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

In [96]:
df3.loc[df3['DPF유무_수정'].isnull() == True, DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
0,경유,승용,소형,2007,16,1.50,1.50,1.50,1.20,0.00,0.00,0.00
1,경유,화물,소형,2011,12,1.35,1.35,1.35,1.14,0.00,0.00,0.00
2,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,0.00,0.00,0.00
3,경유,화물,소형,2009,14,1.45,1.45,1.45,1.18,0.00,0.00,0.00
4,경유,화물,소형,2008,15,1.50,1.50,1.50,1.20,0.00,0.00,0.00


## ❗ 배출계수(EFi)
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1
- 조건 수정(2023.04.20, 최)
    - 특수 중형 = 승합 중형
    - 화물 중형 = 승합 중형
    - 화물 대형 = 승합 대형
    - 특수 중형 = 승용 중형

In [97]:
EFi_col = ['차종', '차종유형', '연료', '차량연식']
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [98]:
df3[EFi_col].values[0]

array(['승용', '소형', '경유', 2007], dtype=object)

In [99]:
df3['차종유형'].unique()

array(['소형', '중형', '대형', '경형'], dtype=object)

In [100]:
# about 7.9s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []

for car_type, car_size, fuel, car_birth in tqdm(df3[EFi_col].values):
    if fuel == '휘발유':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 8.6275 * (V**-1.0722)
                EFi_NOx = 3.2294 * (V**-0.5763)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 5.1835 * (V**-1.1889)
                EFi_NOx = 1.7525 * (V**-0.6481)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 0.7446 * (V**-0.9392)
                EFi_NOx = 0.3403 * (V**-0.5455)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 45:
                    EFi_CO = 4.9952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 45:
                    EFi_CO = 4.5956 * (V**-0.8461)
                else:
                    EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                EFi_HC = 0.2662 * (V**-0.7830)
                EFi_NOx = 0.4476 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 45:
                    EFi_CO = 4.4517 * (V**-0.8461)
                else:
                    EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                EFi_HC = 0.2556 * (V**-0.7830)
                EFi_NOx = 0.4353 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 45:
                    EFi_CO = 4.3079 * (V**-0.8461)
                else:
                    EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                EFi_HC = 0.2449 * (V**-0.7830)
                EFi_NOx = 0.4230 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.164 * (V**-0.8461)
                else:
                    EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                EFi_HC = 0.2343 * (V**-0.7830)
                EFi_NOx = 0.4106 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 111.67 * (V**-1.1566)
                EFi_HC = 32.017 * (V**-1.4171)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 22.356 * (V**-0.9068)
                EFi_HC = 0.8428 * (V**-0.8829)
                EFi_NOx = 1.2613 * (V**-0.3873)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 1.4898 * (V**-0.3837)
                EFi_HC = 0.1738 * (V**-0.7268)
                EFi_NOx = 0.1563 * (V**-0.2671)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') and (car_size == '대형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 65.4:
                    EFi_CO = 1.4082 * (V**-0.7728)
                else:
                    EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 1.2955 * (V**-0.7728)
                else:
                    EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 65.4:
                    EFi_CO = 1.2550 * (V**-0.7728)
                else:
                    EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 65.4:
                    EFi_CO = 1.2144 * (V**-0.7728)
                else:
                    EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 1.1739 * (V**-0.7728)
                else:
                    EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 11.249 * (V**-0.6579)
                else:
                    EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 16.965 * (V**-0.8461)
                else:
                    EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                EFi_HC = 3.0285 * (V**-0.7830)
                EFi_NOx = 1.9923 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 1.8928 * (V**-0.7830)
                EFi_NOx = 1.2352 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 0.9227 * (V**-0.7830)
                EFi_NOx = 3.8859 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.4952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2000:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2001 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 11.920 * (V**-1.5356)
                EFi_NOx = 3.0649 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 37.573 * (V**-1.2078)
                EFi_HC = 3.1786 * (V**-1.5356)
                EFi_NOx = 1.4931 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 32.899 * (V**-1.2078)
                EFi_HC = 2.7387 * (V**-1.5356)
                EFi_NOx = 1.1808 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') and ((car_size == '중형') or (car_size == '대형')) ) or ( (car_type == '화물') and (car_size == '소형') ):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 36.578 * (V**-1.2078)
                EFi_HC = 3.0337 * (V**-1.5356)
                EFi_NOx = 2.0104 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 14.202 * (V**-1.2078)
                EFi_HC = 1.2233 * (V**-1.5356)
                EFi_NOx = 0.2493 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '화물') and ( (car_size == '중형') or (car_size == '대형') ):
            if 1993 <= car_birth:
                if V <= 65:
                    EFi_CO = 70
                    EFi_HC = 7
                    EFi_NOx = 4.5
                else:
                    EFi_CO = 55
                    EFi_HC = 3.5
                    EFi_NOx = 7.5
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
    elif fuel == '경유':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '중형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.6930 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5414 * (V**-0.7524)
                EFi_HC = 0.0927 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0396 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '대형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.9609 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.7507 * (V**-0.7524)
                EFi_HC = 0.1554 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0361 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 0.9835 * (V**-0.5096)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 1.1412 * (V**-0.4324)
                EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
            elif 1991 <= car_birth <= 1995:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.6313 * (V**-0.7298)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5999 * (V**-0.3294)
                EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
            elif 1996 <= car_birth <= 1997:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6408 * (V**-0.3596)
                EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5168 * (V**-0.3596)
                EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2894 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                if V <= 65.4:
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '승합') and (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 32.550 * (V**-0.4944)
                EFi_HC = 15.753 * (V**-0.5912)
                if V < 80:
                    EFi_NOx = 40.692 * (V**-0.5590)
                else:
                    EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                EFi_PM10 = 5.4886 * (V**-0.5911)
                EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 22.804 * (V**-0.4660)
                else:
                    EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 25.708 * (V**-0.4772)
                else:
                    EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 16.378 * (V**-0.5340)
                EFi_HC = 5.8477 * (V**-0.5466)
                if V < 80:
                    EFi_NOx = 25.436 * (V**-0.4656)
                else:
                    EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                EFi_PM10 = 1.2848 * (V**-0.4715)
                EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                EFi_NOx = 15.001 * (V**-0.4528)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                    EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                    EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
        elif (car_type == '승합') and (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 28.205 * (V**-0.5337)
                EFi_HC = 6.1146 * (V**-0.4979)
                EFi_NOx = 41.346 * (V**-0.3645)
                EFi_PM10 = 5.2158 * (V**-0.5048)
                EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 2.4911 * (V**-0.4149)
                EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 1.4432 * (V**-0.3870)
                EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
            elif car_birth == 2001:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 0.9375 * (V**-0.3910)
                EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 1.1507 * (V**-0.4804)
                EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 9.6452 * (V**-0.5291)
                EFi_HC = 3.2339 * (V**-0.7436)
                EFi_NOx = 30.5870 * (V**-0.3548)
                if V <= 80:
                    EFi_PM10 = 0.4657 * (V**-0.5634)
                    EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                else:
                    EFi_PM10 = 0.0014 * (V**0.7970)
                    EFi_PM2_5 = k * 0.0014 * (V**0.7970)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 11.4415 * (V**-0.8036)
                EFi_HC = 0.6774 * (V**-0.8321)
                EFi_NOx = 112.1229 * (V**-1.6393)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
        elif (car_type == '화물') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 4.5854 * (V**-0.3613)
                EFi_HC = 0.4840 * (V**-0.2756)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.8117 * (V**-0.4071)
                EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
            elif 1991 <= car_birth <= 1995:
                EFi_CO = 3.4774 * (V**-0.3483)
                EFi_HC = 0.4844 * (V**-0.3288)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6188 * (V**-0.4540)
                EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 3.3934 * (V**-0.3837)
                EFi_HC = 0.4955 * (V**-0.3393)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.7037 * (V**-0.5357)
                EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6157 * (V**-0.5357)
                EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.4838 * (V**-0.5357)
                EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth <= 2016:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                    EFi_NOx = 0.0271 * (V**0.7596)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif ( (car_type == '화물') and (car_size == '중형') ) | (car_type == '특수'):
            if car_birth <= 1995:
                EFi_CO = 16.769 * (V**-0.3772)
                EFi_HC = 6.7755 * (V**-0.5003)
                EFi_NOx = 24.915 * (V**-0.3942)
                EFi_PM10 = 3.6772 * (V**-0.5514)
                EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
            elif 1996 <= car_birth <= 2000:
                EFi_CO = 21.057 * (V**-0.4958)
                EFi_HC = 6.7532 * (V**-0.5711)
                EFi_NOx = 25.022 * (V**-0.4240)
                EFi_PM10 = 3.5285 * (V**-0.5962)
                EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
            elif car_birth == 2001:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.4444 * (V**-0.4824)
                EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.0432 * (V**-0.4992)
                EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.0002 * V - 0.00756
                    EFi_PM2_5 = k * 0.0002 * V - 0.00756
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 0.000168 * V - 0.00516
                    EFi_PM2_5 = k * 0.000168 * V - 0.00516
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
        elif (car_type == '화물') and (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 30.402 * (V**-0.4685)
                EFi_HC = 15.75 * (V**-0.582)
                EFi_NOx = 117.49 * (V**-0.365)
                EFi_PM10 = 7.6212 * (V**-0.4183)
                EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 6.0264 * (V**-0.4627)
                EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 4.873 * (V**-0.4382)
                EFi_PM2_5 = k * 4.873 * (V**-0.4382)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 28.399 * (V**-0.5999)
                EFi_HC = 10.031 * (V**-0.5828)
                EFi_NOx = 85.301 * (V**-0.4023)
                EFi_PM10 = 3.7541 * (V**-0.4055)
                EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 52.136 * (V**-0.8618)
                EFi_HC = 3.7878 * (V**-0.5425)
                EFi_NOx = 107.5 * (V**-0.5679)
                EFi_PM10 = 2.6847 * (V**-0.6112)
                EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 4.3762 * (V**-0.4550)
                EFi_HC = 0.3627 * (V**-0.7071)
                EFi_NOx = 18.0405 * (V**-1.0986)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
    elif fuel == 'LPG(액화석유가스)':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 101.79 * (V**-1.6823)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 11.173 * (V**-1.3927)
                EFi_NOx = 7.5371 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 11.173 * (V**-1.3927)
                EFi_NOx = 4.7108 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 3.2821 * (V**-1.3927)
                EFi_NOx = 4.7108 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                EFi_CO = 39.362 * (V**-1.0085)
                EFi_HC = 2.8981 * (V**-1.3927)
                EFi_NOx = 1.8419 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                EFi_CO = 73.022 * (V**-1.2078)
                EFi_HC = 4.4166 * (V**-1.5356)
                EFi_NOx = 2.0280 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth == 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 6.1701 * (V**-1.0719)
                    EFi_HC = 2.3782 * (V**-1.9979)
                else:
                    EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                    EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                if V <= 97.3:
                    EFi_NOx = 0.4758 * (V**-1.0665)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        # 최 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
        elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 16.491 * (V**-1.4713)
                    EFi_HC = 1.9218 * (V**-1.8418)
                else:
                    EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                    EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                if V <= 97.3:
                    EFi_NOx = 0.1246 * (V**-0.441)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
    else:
        # 값 고민 : 1
        # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_CO)
    EFi_HC_list.append(EFi_HC)
    EFi_NOx_list.append(EFi_NOx)
    EFi_PM10_list.append(EFi_PM10)
    EFi_PM2_5_list.append(EFi_PM2_5)

100%|██████████| 366911/366911 [00:01<00:00, 241704.17it/s]


In [101]:
df3.shape[0]

366911

In [102]:
len(EFi_CO_list), len(EFi_HC_list), len(EFi_NOx_list), len(EFi_PM10_list), len(EFi_PM2_5_list)

(366911, 366911, 366911, 366911, 366911)

In [103]:
df3['EFi_CO'] = EFi_CO_list
df3['EFi_HC'] = EFi_HC_list
df3['EFi_NOx'] = EFi_NOx_list
df3['EFi_PM10'] = EFi_PM10_list
df3['EFi_PM2_5'] = EFi_PM2_5_list

In [104]:
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

## 배출량 계산

In [105]:
DF_col + check_DF_col + check_R_col

['연료',
 '차종',
 '차종유형',
 '차량연식',
 '차령',
 'DF_CO',
 'DF_HC',
 'DF_PM',
 'DF_NOx',
 'R_CO',
 'R_HC',
 'R_PM']

In [106]:
df3[DF_col + check_DF_col + check_R_col + check_EFi_col].head()

,연료,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5
0,경유,승용,소형,2007,16,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.03,0.01,1.33,0.01,0.01
1,경유,화물,소형,2011,12,1.35,1.35,1.35,1.14,0.00,0.00,0.00,0.02,0.01,1.33,0.04,0.04
2,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.03,0.01,1.33,0.01,0.01
3,경유,화물,소형,2009,14,1.45,1.45,1.45,1.18,0.00,0.00,0.00,0.02,0.01,1.33,0.04,0.04
4,경유,화물,소형,2008,15,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.02,0.01,1.33,0.04,0.04


In [107]:
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df3['E_HOT_CO'] = df3['VKT'] * (df3['EFi_CO'] / 1000) * df3['DF_CO'] * (1 - df3['R_CO'] / 100)
df3['E_HOT_HC'] = df3['VKT'] * (df3['EFi_HC'] / 1000) * df3['DF_HC'] * (1 - df3['R_HC'] / 100)
df3['E_HOT_NOx'] = df3['VKT'] * (df3['EFi_NOx'] / 1000) * df3['DF_NOx'] * (1 - 0 / 100)
df3['E_HOT_PM10'] = df3['VKT'] * (df3['EFi_PM10'] / 1000) * df3['DF_PM'] * (1 - df3['R_PM'] / 100)
df3['E_HOT_PM2_5'] = df3['VKT'] * (df3['EFi_PM2_5'] / 1000) * df3['DF_PM'] * (1 - df3['R_PM'] / 100)

In [108]:
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [109]:
df3[DF_col + check_DF_col + check_R_col + check_EFi_col + check_E_HOT_col].head()

,연료,차종,차종유형,차량연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,...,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5
0,경유,승용,소형,2007,16,1.50,1.50,1.50,1.20,0.00,...,0.03,0.01,1.33,0.01,0.01,0.55,0.10,20.34,0.20,0.19
1,경유,화물,소형,2011,12,1.35,1.35,1.35,1.14,0.00,...,0.02,0.01,1.33,0.04,0.04,0.37,0.25,26.54,0.95,0.87
2,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,0.00,...,0.03,0.01,1.33,0.01,0.01,0.53,0.12,21.21,0.20,0.18
3,경유,화물,소형,2009,14,1.45,1.45,1.45,1.18,0.00,...,0.02,0.01,1.33,0.04,0.04,0.45,0.30,31.49,1.17,1.07
4,경유,화물,소형,2008,15,1.50,1.50,1.50,1.20,0.00,...,0.02,0.01,1.33,0.04,0.04,0.33,0.22,22.67,0.85,0.79


In [110]:
df3.shape

(366911, 56)

# 자동차-엔진 미가열(Cold-start) 배출

## 배출량 식
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

- $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
- $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
- $N_j$ : 차종 $j$의 수
- $M_j$ : 차종 $j$의 주행거리
- $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
- $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

## 베타($\beta$)
- 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
    - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
    - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

    - 1회 주행거리 : 12.35 km (2002년 기준)
    - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
    - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

- 산정식

-|Factor $\beta$의 산정식
-|-
$Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
$Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

- $T_a$ : 대기온도
- $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)

In [111]:
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온

In [112]:
df3['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df3['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [113]:
df3['Beta(Estimated)'].head()

0   0.28
1   0.28
2   0.28
3   0.28
4   0.28
Name: Beta(Estimated), dtype: float64

In [114]:
df3['Beta(Estimated)'].isnull().sum()

0

## $e^{COLD} / e^{HOT}$

In [115]:
df3['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)'], dtype=object)

In [116]:
df3['차량연식'].describe()

count   366911.00
mean      2004.69
std          5.40
min       1987.00
25%       2000.00
50%       2007.00
75%       2009.00
max       2018.00
Name: 차량연식, dtype: float64

In [117]:
# 휘발유(자동 제어)
df3.loc[(df3['연료'] == '휘발유') & (df3['차량연식'] < 1991) & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df3.loc[(df3['연료'] == '휘발유') & (df3['차량연식'] >= 1991) & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df3.loc[(df3['연료'] == '휘발유') & (df3['차량연식'] < 1991) & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df3.loc[(df3['연료'] == '휘발유') & (df3['차량연식'] >= 1991) & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df3.loc[(df3['연료'] == '휘발유') & (df3['차량연식'] < 1991) & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df3.loc[(df3['연료'] == '휘발유') & (df3['차량연식'] >= 1991) & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df3.loc[df3['연료'] == '휘발유', 'eCOLD/eHOT_PM'] = 1
# 경유
df3.loc[(df3['연료'] == '경유') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df3.loc[(df3['연료'] == '경유') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df3.loc[(df3['연료'] == '경유') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df3.loc[(df3['연료'] == '경유') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta
# LPG(액화석유가스)
df3.loc[(df3['연료'] == 'LPG(액화석유가스)') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df3.loc[(df3['연료'] == 'LPG(액화석유가스)') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df3.loc[(df3['연료'] == 'LPG(액화석유가스)') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df3.loc[(df3['연료'] == 'LPG(액화석유가스)') & ( (df3['차종'] == '승용') | ( (df3['차종'] == '승합') & ((df3['차종유형'] == '경형') | (df3['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1

In [118]:
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df3[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     104981
eCOLD/eHOT_NOx    104981
eCOLD/eHOT_HC     104981
eCOLD/eHOT_PM     104339
dtype: int64

In [119]:
df3[check_ecoldhot_col] = df3[check_ecoldhot_col].fillna(1)
df3[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     0
eCOLD/eHOT_NOx    0
eCOLD/eHOT_HC     0
eCOLD/eHOT_PM     0
dtype: int64

In [120]:
df3.loc[df3['eCOLD/eHOT_CO'] < 0, 'eCOLD/eHOT_CO'].shape

(0,)

In [121]:
df3.loc[df3['eCOLD/eHOT_NOx'] < 0, 'eCOLD/eHOT_NOx'].shape

(0,)

In [122]:
df3.loc[df3['eCOLD/eHOT_HC'] < 0, 'eCOLD/eHOT_HC'].shape

(0,)

In [123]:
df3.loc[df3['eCOLD/eHOT_PM'] < 0, 'eCOLD/eHOT_PM'].shape

(0,)

## 배출량 계산

In [124]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '1일평균주행거리', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'DPF유무_수정', 'R_CO', 'R_HC', 'R_PM',
       'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO',
       'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM'],
      dtype='object')

### 🔺 계산식 고민
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
- $M_j$ : 차종 $j$의 주행거리
    - 1년 주행거리로 계산? 
        - VKT
    - 1회 주행거리로 계산?
        - l_trip

In [125]:
#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df3['E_COLD_CO'] = df3['Beta(Estimated)'] * df3['VKT'] * df3['EFi_CO'] * (df3['eCOLD/eHOT_CO'] - 1)
df3['E_COLD_NOx'] = df3['Beta(Estimated)'] * df3['VKT'] * df3['EFi_NOx'] * (df3['eCOLD/eHOT_NOx'] - 1)
df3['E_COLD_HC'] = df3['Beta(Estimated)'] * df3['VKT'] * df3['EFi_HC'] * (df3['eCOLD/eHOT_HC'] - 1)
df3['E_COLD_PM10'] = df3['Beta(Estimated)'] * df3['VKT'] * df3['EFi_PM10'] * (df3['eCOLD/eHOT_PM'] - 1)
df3['E_COLD_PM2_5'] = df3['Beta(Estimated)'] * df3['VKT'] * df3['EFi_PM2_5'] * (df3['eCOLD/eHOT_PM'] - 1)

In [126]:
df3.loc[df3['E_COLD_NOx'] < 0, ['연료', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].shape

(5589, 6)

In [127]:
df3.loc[df3['E_COLD_NOx'] < 0, ['연료', 'Beta(Estimated)', 'VKT', 'EFi_NOx', 'eCOLD/eHOT_NOx', 'E_COLD_NOx']].head()

,연료,Beta(Estimated),VKT,EFi_NOx,eCOLD/eHOT_NOx,E_COLD_NOx
145,LPG(액화석유가스),0.28,12994.00,0.41,0.91,-137.33
285,LPG(액화석유가스),0.28,12957.50,0.35,0.91,-117.78
342,LPG(액화석유가스),0.28,14125.50,0.35,0.91,-128.39
549,LPG(액화석유가스),0.28,13286.00,0.41,0.91,-140.42
668,LPG(액화석유가스),0.28,12884.50,0.41,0.91,-136.18


In [128]:
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']
df3[check_E_HOT_col + check_E_cold_col].head()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
0,0.55,0.10,20.34,0.20,0.19,53.16,650.26,18.54,32.39,29.80
1,0.37,0.25,26.54,0.95,0.87,0.00,0.00,0.00,0.00,0.00
2,0.53,0.12,21.21,0.20,0.18,51.99,678.29,21.73,31.86,29.31
3,0.45,0.30,31.49,1.17,1.07,0.00,0.00,0.00,0.00,0.00
4,0.33,0.22,22.67,0.85,0.79,0.00,0.00,0.00,0.00,0.00


In [129]:
df3[check_E_HOT_col + check_E_cold_col].tail()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
366906,30.70,2.31,9.55,0.01,0.01,29381.99,3414.08,3463.01,0.00,0.00
366907,0.42,0.28,30.11,1.10,1.01,0.00,0.00,0.00,0.00,0.00
366908,30.70,2.31,9.55,0.01,0.01,29381.99,3414.08,3463.01,0.00,0.00
366909,0.57,0.13,22.73,0.22,0.20,55.71,726.87,23.28,34.14,31.41
366910,10.39,2.65,90.63,0.19,0.17,0.00,0.00,0.00,0.00,0.00


In [130]:
df3[check_E_cold_col].isnull().sum()

E_COLD_CO       0
E_COLD_NOx      0
E_COLD_HC       0
E_COLD_PM10     0
E_COLD_PM2_5    0
dtype: int64

# 자동차-휘발유 증발 배출

## 배출량 식
- 3가지
    - 주간증발손실(Diurnal loss)
    - 고온증발손실(Hot and warm soak)
    - 주행손실(Running loss)
- 현재('19 배출량 기준) 휘발유 증발 배출원
    - 주간증발손실, 주행손실 고려
    - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

---

$$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

$S^{fi} = q \times e^{fi} \times x$
$$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

- $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
- $m_j$ : 차종 j의 연간 총 주행거리
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

## 배출계수

### 🔺 수정 중

### $e^d$ Diurnal(g/day)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)

#### 🔺 RVP(증기압)
- 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
    - summer : 60
    - winter : 70
    - 평균 : 65
- 수정 사항(2023.04.20, 최)
    - 국내 과학기술원, 환경부 자료 찾아서 사용 추천

In [131]:
RVP = 65

#### ❗ t_a(기온)
- t_a, t_min, t_rise
    - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
        - t_a = (t_max + t_min) / 2
        - t_rise = t_max - t_min
    - 기상자료개방포털(2002년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)

In [132]:
t_min = 7.5 # 평균 최저 온도
t_max = 17.9
t_a = (t_min + t_max) / 2
t_rise = 12.4 # 평균 상승 온도

In [133]:
df3.loc[df3['차량연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df3.loc[df3['차량연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [134]:
df3['e_d'].isnull().sum()

0

### $e^{R,HOT}$ Hot running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [135]:
df3.loc[df3['차량연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df3.loc[df3['차량연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [136]:
df3['e_RHOT'].isnull().sum()

0

### $e^{S,HOT}$ Hot soak(g/procedure)
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수

In [137]:
df3.loc[df3['차량연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df3.loc[df3['차량연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

### $e^{S,WARM}$ Waram soak(g/procedure)
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [138]:
df3.loc[df3['차량연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df3.loc[df3['차량연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수

In [139]:
df3.loc[df3['차량연식'] < 1991, 'e_fi'] = 0.7
df3.loc[df3['차량연식'] >= 1991, 'e_fi'] = 0

### $e^{R,WARM}$ Warm running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [140]:
df3.loc[df3['차량연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df3.loc[df3['차량연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [141]:
df3['e_RWARM'].isnull().sum()

0

## $R$
$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
- Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $m_j$ : 차종 j의 연간 총 주행거리
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $w$ : 미가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수

### ❓ 확인 중
- 수정 사항(2023.04.20, 최)
    - 찾는 중

In [142]:
p = 0.5 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.5 # 미가열 엔진 상태로 끝나는 trip의 비율

In [143]:
df3['R'] = df3['VKT'] * (p * df3['e_RHOT'] + w * df3['e_RWARM'])

In [144]:
df3['R'].isnull().sum()

0

## $S^{fi}$
$S^{fi} = q \times e^{fi} \times x$
- Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [145]:
not_injection = df3[(df3['연료'] == '휘발유') & (df3['차량연식'] < 1991)].shape[0]
injection = df3[(df3['연료'] == '휘발유') & (df3['차량연식'] >= 1991)].shape[0]
# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)
q

0.986326388388244

In [146]:
# x = df3['VKT'] / (365 * l_trip)
df3['S_fi'] = q * df3['e_fi'] * ( df3['VKT'] / (365 * l_trip) )

## $S^c$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
- Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [147]:
# x = df3['VKT'] / (365 * l_trip)
df3['S_c'] = (1-q) * (p * (df3['VKT'] / (365 * l_trip)) * df3['e_SHOT'] + w * (df3['VKT'] / (365 * l_trip)) * df3['e_SWARM'])

## $E_{EVA,VOC}$
$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)

In [148]:
df3['E_EVA_VOC'] = 365 * df3['e_d'] + df3['S_c'] + df3['S_fi'] + df3['R']

In [149]:
df3['E_EVA_VOC'].isnull().sum()

0

# 배출량 합계
$E_{total}(kg) = E_{HOT}(kg) + E_{COLD}(g) + E_{EVAP}(g)$

In [150]:
df3[df3['E_COLD_NOx'] < 0].shape

(5589, 76)

In [151]:
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df3.loc[df3['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [152]:
df3[df3['E_COLD_NOx'] < 0].shape

(0, 76)

In [153]:
check_E_HOT_col, check_E_cold_col

(['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5'],
 ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5'])

In [154]:
df3.loc[(df3['연료'] == '휘발유') & (df3['E_COLD_PM10'] != 0)].shape

(0, 76)

In [155]:
df3['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)'], dtype=object)

In [156]:
fuel = '휘발유'
df3.loc[df3['연료'] == fuel, 'E_CO_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_CO'] + df3.loc[df3['연료'] == fuel , 'E_COLD_CO']/1000
df3.loc[df3['연료'] == fuel, 'E_HC_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_HC'] + df3.loc[df3['연료'] == fuel , 'E_COLD_HC']/1000 + df3.loc[df3['연료'] == fuel , 'E_EVA_VOC']/1000
df3.loc[df3['연료'] == fuel, 'E_NOx_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_NOx'] + df3.loc[df3['연료'] == fuel , 'E_COLD_NOx']/1000
df3.loc[df3['연료'] == fuel, 'E_PM10_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_PM10'] + df3.loc[df3['연료'] == fuel , 'E_COLD_PM10']/1000
df3.loc[df3['연료'] == fuel, 'E_PM2_5_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_PM2_5'] + df3.loc[df3['연료'] == fuel , 'E_COLD_PM2_5']/1000

In [157]:
fuel = '경유'
df3.loc[df3['연료'] == fuel, 'E_CO_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_CO'] + df3.loc[df3['연료'] == fuel , 'E_COLD_CO']/1000
df3.loc[df3['연료'] == fuel, 'E_HC_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_HC'] + df3.loc[df3['연료'] == fuel , 'E_COLD_HC']/1000
df3.loc[df3['연료'] == fuel, 'E_NOx_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_NOx'] + df3.loc[df3['연료'] == fuel , 'E_COLD_NOx']/1000
df3.loc[df3['연료'] == fuel, 'E_PM10_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_PM10'] + df3.loc[df3['연료'] == fuel , 'E_COLD_PM10']/1000
df3.loc[df3['연료'] == fuel, 'E_PM2_5_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_PM2_5'] + df3.loc[df3['연료'] == fuel , 'E_COLD_PM2_5']/1000

In [158]:
fuel = 'LPG(액화석유가스)'
df3.loc[df3['연료'] == fuel, 'E_CO_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_CO'] + df3.loc[df3['연료'] == fuel , 'E_COLD_CO']/1000
df3.loc[df3['연료'] == fuel, 'E_HC_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_HC'] + df3.loc[df3['연료'] == fuel , 'E_COLD_HC']/1000
df3.loc[df3['연료'] == fuel, 'E_NOx_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_NOx'] + df3.loc[df3['연료'] == fuel , 'E_COLD_NOx']/1000
df3.loc[df3['연료'] == fuel, 'E_PM10_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_PM10'] + df3.loc[df3['연료'] == fuel , 'E_COLD_PM10']/1000
df3.loc[df3['연료'] == fuel, 'E_PM2_5_total'] = df3.loc[df3['연료'] == fuel , 'E_HOT_PM2_5'] + df3.loc[df3['연료'] == fuel , 'E_COLD_PM2_5']/1000

In [159]:
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']
df3[check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,0.60,0.12,20.99,0.24,0.22
1,0.37,0.25,26.54,0.95,0.87
2,0.59,0.14,21.89,0.23,0.21
3,0.45,0.30,31.49,1.17,1.07
4,0.33,0.22,22.67,0.85,0.79


In [160]:
df3.loc[df3['연료'] == '휘발유', check_E_col + ['E_EVA_VOC']].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_EVA_VOC
7,58.13,9.79,11.38,0.01,0.01,2182.05
8,57.95,5.94,12.51,0.01,0.01,370.29
17,53.83,7.34,12.08,0.01,0.01,368.29
18,58.28,5.97,12.58,0.01,0.01,370.63
29,60.14,8.16,13.50,0.01,0.01,374.97


In [161]:
df3.loc[df3['연료'] == '경유', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,0.60,0.12,20.99,0.24,0.22
1,0.37,0.25,26.54,0.95,0.87
2,0.59,0.14,21.89,0.23,0.21
3,0.45,0.30,31.49,1.17,1.07
4,0.33,0.22,22.67,0.85,0.79


In [162]:
df3.loc[df3['연료'] == 'LPG(액화석유가스)', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
71,47.69,3.20,4.76,0.01,0.01
73,45.34,1.44,13.25,0.01,0.01
131,57.79,3.88,5.76,0.01,0.01
138,51.27,3.44,5.11,0.01,0.01
145,63.07,2.97,10.53,0.01,0.00


In [163]:
df3.loc[(df3['연료'] == '경유') & (df3['차종'] == '승용'), DF_col + check_E_col].head()

,연료,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,경유,승용,소형,2007,16,0.60,0.12,20.99,0.24,0.22
2,경유,승용,중형,2008,15,0.59,0.14,21.89,0.23,0.21
5,경유,승용,중형,2007,16,0.58,0.14,21.77,0.23,0.21
9,경유,승용,소형,2007,16,0.64,0.13,22.25,0.25,0.23
10,경유,승용,중형,2008,15,0.60,0.14,22.31,0.24,0.22


In [164]:
df3.loc[(df3['연료'] == '경유') & (df3['차종'] == '화물'), DF_col + check_E_col].head()

,연료,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1,경유,화물,소형,2011,12,0.37,0.25,26.54,0.95,0.87
3,경유,화물,소형,2009,14,0.45,0.30,31.49,1.17,1.07
4,경유,화물,소형,2008,15,0.33,0.22,22.67,0.85,0.79
13,경유,화물,소형,2010,13,0.38,0.26,27.29,0.99,0.91
14,경유,화물,소형,2011,12,0.36,0.24,26.20,0.94,0.86


In [165]:
df3.loc[(df3['연료'] == '휘발유') & (df3['차종'] == '승용'), DF_col + check_E_col].head()

,연료,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
7,휘발유,승용,중형,1989,34,58.13,9.79,11.38,0.01,0.01
8,휘발유,승용,중형,1999,24,57.95,5.94,12.51,0.01,0.01
17,휘발유,승용,소형,1996,27,53.83,7.34,12.08,0.01,0.01
18,휘발유,승용,중형,1998,25,58.28,5.97,12.58,0.01,0.01
29,휘발유,승용,소형,1996,27,60.14,8.16,13.50,0.01,0.01


In [166]:
df3.loc[(df3['연료'] == '휘발유') & (df3['차종'] == '화물'), DF_col + check_E_col].head()

,연료,차종,차종유형,차량연식,차령,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
799,휘발유,화물,경형,1996,27,30.43,6.69,17.18,0.01,0.01
3155,휘발유,화물,소형,1994,29,42.91,3.65,13.47,0.01,0.01
3197,휘발유,화물,소형,1992,31,41.17,3.51,12.92,0.01,0.01
4490,휘발유,화물,소형,1991,32,25.70,2.31,8.07,0.01,0.01
4521,휘발유,화물,경형,1997,26,53.34,5.51,16.20,0.01,0.01


## issue
- LPG : E_COLD_NOx 값 음수 -> 0으로 처리

In [167]:
df3.loc[df3['연료'] == 'LPG(액화석유가스)', DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,차량연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
71,LPG(액화석유가스),화물,경형,2002,21,47.69,3.20,4.76,0.01,0.01,0.00,0.00,0.00,0.00,0.00,47.69,3.20,4.76,0.01,0.01
73,LPG(액화석유가스),화물,소형,2002,21,45.34,1.44,13.25,0.01,0.01,0.00,0.00,0.00,0.00,0.00,45.34,1.44,13.25,0.01,0.01
131,LPG(액화석유가스),화물,경형,2002,21,57.79,3.88,5.76,0.01,0.01,0.00,0.00,0.00,0.00,0.00,57.79,3.88,5.76,0.01,0.01
138,LPG(액화석유가스),화물,경형,2000,23,51.27,3.44,5.11,0.01,0.01,0.00,0.00,0.00,0.00,0.00,51.27,3.44,5.11,0.01,0.01
145,LPG(액화석유가스),승용,중형,1999,24,51.98,2.78,10.53,0.01,0.00,11092.07,0.00,190.44,0.00,0.00,63.07,2.97,10.53,0.01,0.00


In [168]:
df3.loc[df3['연료'] == 'LPG(액화석유가스)', DF_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,차량연식,차령,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
71,LPG(액화석유가스),화물,경형,2002,21,0.00,0.00,0.00,0.00,0.00,47.69,3.20,4.76,0.01,0.01
73,LPG(액화석유가스),화물,소형,2002,21,0.00,0.00,0.00,0.00,0.00,45.34,1.44,13.25,0.01,0.01
131,LPG(액화석유가스),화물,경형,2002,21,0.00,0.00,0.00,0.00,0.00,57.79,3.88,5.76,0.01,0.01
138,LPG(액화석유가스),화물,경형,2000,23,0.00,0.00,0.00,0.00,0.00,51.27,3.44,5.11,0.01,0.01
145,LPG(액화석유가스),승용,중형,1999,24,11092.07,0.00,190.44,0.00,0.00,63.07,2.97,10.53,0.01,0.00


In [169]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '1일평균주행거리', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'DPF유무_수정', 'R_CO', 'R_HC', 'R_PM',
       'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO',
       'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_total', 'E_PM2_5_total'],
      dtype='object')

In [170]:
df3['E_COLD_CO'].dtype

dtype('float64')

In [171]:
df3.loc[df3['E_CO_total'] < 0].shape

(0, 81)

In [172]:
df3.loc[df3['E_HC_total'] < 0].shape

(0, 81)

In [173]:
df3.loc[df3['E_PM10_total'] < 0].shape

(0, 81)

In [174]:
df3.loc[df3['E_PM2_5_total'] < 0].shape

(0, 81)

In [175]:
df3.loc[df3['E_NOx_total'] < 0, ['E_NOx_total', 'E_COLD_NOx']].shape

(0, 2)

## [출력] 배출량
- 연료 : 휘발유, 경유, LPG(액화석유가스)
- 물질 : CO, HC, NOx, PM10, PM2.5

In [176]:
df3.columns

Index(['차량관리번호', '본거지법정동코드', '차종', '차대번호', '최초등록일자', '차량연식', '제원관리번호',
       '배출가스인증번호', '배출가스등급', '차량말소YN_등록정보', '차명', '차종유형', '제작사명', '연료', '등급',
       '차량말소YN_변경이력', '변경일자', '변경구분코드', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '저감장치', '변경일자_년', '변경일자_월',
       '변경일자_일', '변경일자_년_월', '차령', '시구군_수정', '시도명', '시군구명', '1일평균주행거리', 'VKT',
       'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'DPF유무_수정', 'R_CO', 'R_HC', 'R_PM',
       'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO',
       'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)',
       'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM',
       'E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5',
       'e_d', 'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi',
       'S_c', 'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total',
       'E_PM10_total', 'E_PM2_5_total'],
      dtype='object')

In [177]:
df3.loc[df3['변경일자_년'] == 2022].shape

(54713, 81)

In [178]:
df3.shape

(366911, 81)

In [179]:
df3.loc[df3['시구군_수정'].str.split(' ').str.len() != 1, '시구군_수정'].head()

Series([], Name: 시구군_수정, dtype: object)

# 연료/시도 별 배출량 합계

In [181]:
grp = df3.loc[df3['변경일자_년'] == 2022].groupby(['시도명', '시구군_수정']).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum'}).reset_index()
grp

,시도명,시구군_수정,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,강원도,강릉시,3778.51,496.55,7421.41,86.38,79.47
1,강원도,고성군,437.55,71.07,1257.71,15.83,14.57
2,강원도,동해시,1406.44,215.86,2445.46,25.94,23.87
3,강원도,삼척시,1180.81,147.17,2291.10,21.99,20.23
4,강원도,속초시,2279.82,289.90,2690.70,26.95,24.80
...,...,...,...,...,...,...,...
248,충청북도,제천시,1791.66,254.25,5450.80,73.99,68.07
249,충청북도,증평군,312.84,37.86,1234.00,15.17,13.96
250,충청북도,진천군,962.14,129.12,3756.23,48.42,44.55
251,충청북도,청주시,12245.25,1599.39,31098.21,357.13,328.56


In [182]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.05_PM.11.31.44'

In [184]:
grp.to_excel(os.path.join(analysis_fold, f'BD1-3_통계_4등급_2022년말소차량_시도별_CAPSS매연량합계.xlsx'), sheet_name='시도', index=False)

# 코드 마지막